*Farzaneh Soltanzadeh*

In [2]:
from sqlalchemy import create_engine
from sqlalchemy import URL
from sqlalchemy import text
from urllib.parse import quote_plus
from sqlalchemy.types import Integer, VARCHAR
import pandas as pd 

In [3]:
# !cd D:\University\Quera-BootCamp\

In [ ]:
# !jupytext --to py:percent -o db.py db.ipynb

 Connection and Create DataBase

In [89]:
url_object = URL.create(
    "mysql+mysqlconnector",
    username="root",
    password="farzansql",
    host="localhost",
)

In [90]:
engine = create_engine(url_object)

In [91]:
with engine.connect() as conn : 
    conn.execute(text('CREATE Schema imdb250;'))
    # conn.execute(text('DROP schema imdb250;'))

Read Data from Json file

In [3]:
df_movies = pd.read_json('imdb_250_movies.json', dtype={'id': 'str'})
df_movies.head()

,id,title,year,parental_guide,runtime,genre,directore,writer,star,gross_us_canada
0,0111161,The Shawshank Redemption,1994,R,142,[Drama],[Frank Darabont],"[Stephen King, Frank Darabont]","[Tim Robbins, Morgan Freeman, Bob Gunton]",28767189.0
1,0068646,The Godfather,1972,R,175,"[Crime, Drama]",[Francis Ford Coppola],"[Mario Puzo, Francis Ford Coppola]","[Marlon Brando, Al Pacino, James Caan]",136381073.0
2,0468569,The Dark Knight,2008,PG-13,152,"[Action, Crime, Drama]",[Christopher Nolan],"[Jonathan Nolan, Christopher Nolan, David S. G...","[Christian Bale, Heath Ledger, Aaron Eckhart]",534987076.0
3,0071562,The Godfather Part II,1974,R,202,"[Crime, Drama]",[Francis Ford Coppola],"[Francis Ford Coppola, Mario Puzo]","[Al Pacino, Robert De Niro, Robert Duvall]",47834595.0
4,0050083,12 Angry Men,1957,Approved,96,"[Crime, Drama]",[Sidney Lumet],[Reginald Rose],"[Henry Fonda, Lee J. Cobb, Martin Balsam]",NaN


In [4]:
df_persons = pd.read_json('imdb_persons_id.json', dtype={'id': 'str'}).drop_duplicates().reset_index(drop=True)
df_persons.head()

,name,id
0,Frank Darabont,0001104
1,Stephen King,0000175
2,Tim Robbins,0000209
3,Morgan Freeman,0000151
4,Bob Gunton,0348409


CREATE TABLE movie

In [94]:
df_movie_table = df_movies.loc[:, ['id', 'title', 'year', 'runtime', 'parental_guide', 'gross_us_canada']].set_index('id')
df_movie_table.head()

,title,year,runtime,parental_guide,gross_us_canada
id,,,,,
0111161,The Shawshank Redemption,1994,142,R,28767189.0
0068646,The Godfather,1972,175,R,136381073.0
0468569,The Dark Knight,2008,152,PG-13,534987076.0
0071562,The Godfather Part II,1974,202,R,47834595.0
0050083,12 Angry Men,1957,96,Approved,NaN


In [95]:
with engine.connect() as conn:
    df_movie_table.to_sql(name='movie', con=conn, schema='imdb250', if_exists='replace', chunksize=50, method='multi',
                          index_label= 'id', dtype={'id': VARCHAR(8), 'title': VARCHAR(128), 'year': Integer(), 'runtime': Integer(), 'parental_guide': VARCHAR(8), 'gross_us_canada': Integer()})


CREATE TABLE person

In [96]:
df_peron_table = df_persons.loc[:, ['id', 'name']].set_index('id')
df_peron_table.head()

,name
id,
0001104,Frank Darabont
0000175,Stephen King
0000209,Tim Robbins
0000151,Morgan Freeman
0348409,Bob Gunton


In [97]:
with engine.connect() as conn:
    df_peron_table.to_sql(name='person', con=conn, schema='imdb250', if_exists='replace', chunksize=50, method='multi',
                          index_label= 'id', dtype={'id': VARCHAR(8), 'name': VARCHAR(32)})


CREATE TABLE cast

In [98]:
qu_cast = text("""
CREATE TABLE cast (
id INT NOT NULL AUTO_INCREMENT,
movie_id VARCHAR(8) NOT NULL,
person_id VARCHAR(8) NOT NULL,
PRIMARY KEY (id),
FOREIGN KEY (movie_id) REFERENCES movie(id),
FOREIGN KEY (person_id) REFERENCES person(id)
);""")

In [99]:
with engine.connect() as conn:
    conn.execute(text('USE imdb250;'))
    conn.execute(qu_cast)

In [100]:
list_cast_table = []

for index, row in df_movies.loc[:, ['id', 'star']].iterrows():
    for name in row['star']:
        list_cast_table.append({'movie_id': row['id'], 'person_id': df_persons.loc[df_persons['name'] == name, 'id'].tolist()[0]})
        
df_cast_table = pd.DataFrame(list_cast_table).set_index('movie_id')
df_cast_table.head()

,person_id
movie_id,
0111161,0000209
0111161,0000151
0111161,0348409
0068646,0000008
0068646,0000199


In [101]:
with engine.connect() as conn:
    df_cast_table.to_sql(name='cast', con=conn, schema='imdb250', if_exists='append', chunksize=50, method='multi',
                          index=True, dtype={'movie_id': VARCHAR(8), 'person_id': VARCHAR(8)})

CREATE TABLE crew

In [102]:
qu_crew = text("""
CREATE TABLE crew (
id INT NOT NULL AUTO_INCREMENT,
movie_id VARCHAR(8) NOT NULL,
person_id VARCHAR(8) NOT NULL,
role VARCHAR(8),
PRIMARY KEY (id),
FOREIGN KEY (movie_id) REFERENCES movie(id),
FOREIGN KEY (person_id) REFERENCES person(id)
);""")

In [103]:
with engine.connect() as conn:
    conn.execute(text('USE imdb250;'))
    conn.execute(qu_crew)

In [104]:
list_crew_table = []

for index, row in df_movies.loc[:, ['id', 'directore', 'writer']].iterrows():
    for name in row['directore']:
        list_crew_table.append({'movie_id': row['id'], 'person_id': df_persons.loc[df_persons['name'] == name, 'id'].tolist()[0], 'role': 'Director'})
    for name in row['writer']:
        list_crew_table.append({'movie_id': row['id'], 'person_id': df_persons.loc[df_persons['name'] == name, 'id'].tolist()[0], 'role': 'Writer'})
     
df_crew_table = pd.DataFrame(list_crew_table).set_index('movie_id')
df_crew_table.head()

,person_id,role
movie_id,,
0111161,0001104,Director
0111161,0000175,Writer
0111161,0001104,Writer
0068646,0000338,Director
0068646,0701374,Writer


In [105]:
with engine.connect() as conn:
    df_crew_table.to_sql(name='crew', con=conn, schema='imdb250', if_exists='append', chunksize=50, method='multi',
                          index=True, dtype={'movie_id': VARCHAR(8), 'person_id': VARCHAR(8), 'role': VARCHAR(8)})

CREATE TABLE genre

In [106]:
qu_genre = text("""
CREATE TABLE genre (
id INT NOT NULL AUTO_INCREMENT,
movie_id VARCHAR(8),
genre VARCHAR(16),
PRIMARY KEY (id),
FOREIGN KEY (movie_id) REFERENCES movie(id)
);""")

In [107]:
with engine.connect() as conn:
    conn.execute(text('USE imdb250;'))
    conn.execute(qu_genre)

In [108]:
list_genre_table = []

for index, row in df_movies.loc[:, ['id', 'genre']].iterrows():
    for genre in row['genre']:
        list_genre_table.append({'movie_id': row['id'], 'genre': genre})
   
df_genre_table = pd.DataFrame(list_genre_table).set_index('movie_id')
df_genre_table.head()

,genre
movie_id,
0111161,Drama
0068646,Crime
0068646,Drama
0468569,Action
0468569,Crime


In [109]:
with engine.connect() as conn:
    df_genre_table.to_sql(name='genre', con=conn, schema='imdb250', if_exists='append', chunksize=50, method='multi',
                          index=True, dtype={'movie_id': VARCHAR(8), 'genre': VARCHAR(16)})